In [15]:
import os
import pandas as pd

# 'data/raw_data' 경로 설정
root_dir = 'data/'
visitor_city = pd.read_csv(os.path.join(root_dir, "city_of_festival_visitors.csv"))
address = pd.read_csv(os.path.join(root_dir, "address_with_lon_lat_final.csv"))
mooju = set(list(address[address['시군구명'] == '무주군']['행정동코드']))
other_city = list(address.merge(visitor_city, on=['시도명', '시군구명'])['행정동코드_x'])

In [16]:
import os
import pandas as pd

od_df = pd.DataFrame()
raw_data_dir = os.path.join(root_dir, "raw_data/")
output_file_path = os.path.join(root_dir, "od_df.csv")  # od_df.csv 파일 경로

# 전처리코드는 모두 여기에
def preprocess(df):
    # dest_hdong_cd 값이 축제가 열리는 곳인 무주군 데이터만 필터링
    #filtered_df = df[df['dest_hdong_cd'].isin(mooju)]

    # origin_hdong_cd 값이 visitor_city 인 데이터만 필터링
    #filtered_df = filtered_df[filtered_df['origin_hdong_cd'].isin(other_city)]

    """# 타 지역에서 온 데이터만 필터링
    filtered_df = filtered_df[~filtered_df['origin_hdong_cd'].isin(mooju)]"""
    filtered_df = df
    return filtered_df

# od_df.csv 파일이 존재하는지 확인
if os.path.exists(output_file_path):
    print(f"'{output_file_path}' 파일이 이미 존재합니다. 실행을 종료합니다.")
else:
    # 'od'로 시작하는 폴더 내의 모든 CSV 파일 처리
    for dirpath, dirnames, filenames in os.walk(raw_data_dir):
        if os.path.basename(dirpath).startswith('od'):
            for filename in filenames:
                if filename.endswith('.csv'):
                    # 파일 이름에서 날짜 추출
                    date_str = filename.split('_')[1]

                    # 파일 경로 설정 및 CSV 읽기
                    file_path = os.path.join(dirpath, filename)
                    csv_data = pd.read_csv(file_path)

                    # 전처리
                    filtered_data = preprocess(csv_data)

                    # 날짜에서 월일(MMDD) 부분 추출
                    mmdd_str = date_str[4:]  # 'YYYYMMDD'에서 마지막 네 자리 'MMDD' 추출

                    # 동적으로 변수 생성 (예: df_0901)
                    globals()[f'df_{mmdd_str}'] = filtered_data

                    # 데이터프레임 이름 출력 (예: df_0901)
                    print(f"데이터프레임 df_{mmdd_str}가 생성되었고, 기존 데이터프레임에 포함합니다.")
                    od_df = pd.concat([od_df, globals()[f'df_{mmdd_str}']])

    # 데이터 처리 완료 후 od_df.csv 파일로 저장
    od_df.to_csv(output_file_path, index=False)
    print(f"데이터프레임 od_df가 '{output_file_path}'에 저장되었습니다.")


데이터프레임 df_0901가 생성되었고, 기존데이터프레임에 포함합니다.
데이터프레임 df_0902가 생성되었고, 기존데이터프레임에 포함합니다.
데이터프레임 df_0903가 생성되었고, 기존데이터프레임에 포함합니다.
데이터프레임 df_0904가 생성되었고, 기존데이터프레임에 포함합니다.
데이터프레임 df_0905가 생성되었고, 기존데이터프레임에 포함합니다.
데이터프레임 df_0906가 생성되었고, 기존데이터프레임에 포함합니다.
데이터프레임 df_0907가 생성되었고, 기존데이터프레임에 포함합니다.
데이터프레임 df_0908가 생성되었고, 기존데이터프레임에 포함합니다.
데이터프레임 df_0909가 생성되었고, 기존데이터프레임에 포함합니다.
데이터프레임 df_0910가 생성되었고, 기존데이터프레임에 포함합니다.
데이터프레임 df_0911가 생성되었고, 기존데이터프레임에 포함합니다.
데이터프레임 df_0912가 생성되었고, 기존데이터프레임에 포함합니다.
데이터프레임 df_0913가 생성되었고, 기존데이터프레임에 포함합니다.
데이터프레임 df_0914가 생성되었고, 기존데이터프레임에 포함합니다.
데이터프레임 df_0915가 생성되었고, 기존데이터프레임에 포함합니다.
데이터프레임 df_0916가 생성되었고, 기존데이터프레임에 포함합니다.
데이터프레임 df_0917가 생성되었고, 기존데이터프레임에 포함합니다.
데이터프레임 df_0918가 생성되었고, 기존데이터프레임에 포함합니다.
데이터프레임 df_0919가 생성되었고, 기존데이터프레임에 포함합니다.
데이터프레임 df_0920가 생성되었고, 기존데이터프레임에 포함합니다.
데이터프레임 df_0921가 생성되었고, 기존데이터프레임에 포함합니다.
데이터프레임 df_0922가 생성되었고, 기존데이터프레임에 포함합니다.
데이터프레임 df_0923가 생성되었고, 기존데이터프레임에 포함합니다.
데이터프레임 df_0924가 생성되었고, 기존데이터프레임에 포함합니다.
데이터프레임 df_0925가 생성되었고, 기존데이터프레임에 포함합니다.


In [47]:
od_df = od_df[od_df['modal'] == 0]
od_df.head()

,origin_hdong_cd,dest_hdong_cd,date,start_time,end_time,gender,age,modal,origin_purpose,dest_purpose,od_dist_avg,od_duration_avg,od_cnts
0,1130563500,1130559500,2023-09-01,12:00,13:00,1,3,0.0,1.0,1,10869,58,7
1,2714072000,2714073000,2023-09-01,12:00,12:00,1,4,0.0,0.0,0,2018,4,16
2,3017055500,3017058700,2023-09-01,18:00,18:00,1,2,0.0,4.0,0,14070,39,8
3,2917067300,2917059000,2023-09-01,19:00,19:00,0,4,0.0,0.0,0,2738,9,22
5,1165058100,1165058000,2023-09-01,14:00,14:00,1,4,0.0,1.0,1,9067,8,15


In [24]:
visitor_city['시도 시군구'] = visitor_city['시도명'].fillna('') + ' ' + visitor_city['시군구명'].fillna('')

# 양쪽 값이 모두 null인 경우 빈 문자열 처리
visitor_city['시도 시군구'] = visitor_city['시도 시군구'].str.strip()
visitor_city.head()

,시군구명,od_cnts,시도명,행정동코드,위도,경도,시도 시군구
0,전주시 덕진구,2834,전라북도,4511300000,35.847561,127.117672,전라북도 전주시 덕진구
1,서구,1813,대전광역시,3017000000,36.355179,127.383849,대전광역시 서구
2,전주시 완산구,1561,전라북도,4511100000,35.795512,127.132447,전라북도 전주시 완산구
3,유성구,1379,대전광역시,3020000000,36.362073,127.356410,대전광역시 유성구
4,동구,1207,대전광역시,3011000000,35.886500,128.635500,대전광역시 동구


In [25]:
address['시도 시군구'] = address['시도명'].fillna('') + ' ' + address['시군구명'].fillna('')

# 양쪽 값이 모두 null인 경우 빈 문자열 처리
address['시도 시군구'] = address['시도 시군구'].str.strip()
address.head()

,행정동코드,시도명,시군구명,읍면동명,동리명,위도,경도,시도 시군구
0,1100000000,서울특별시,NaN,NaN,서울특별시,37.566679,126.978291,서울특별시
1,1111000000,서울특별시,종로구,NaN,종로구,37.580695,126.982799,서울특별시 종로구
2,1111051500,서울특별시,종로구,청운효자동,세종로,37.579997,126.976930,서울특별시 종로구
3,1111051500,서울특별시,종로구,청운효자동,옥인동,37.583480,126.963850,서울특별시 종로구
4,1111051500,서울특별시,종로구,청운효자동,누하동,37.578998,126.967561,서울특별시 종로구


In [66]:
daejun_shuttle = ['세종특별자치시', '대전광역시 유성구', '대전광역시 서구', '대전광역시 대덕구', '대전광역시 중구', '충청남도 금산군', '충청북도 영동군', '전라북도 무주군']

In [67]:
duration_table = []
distance_table = []

for idx,target in enumerate(daejun_shuttle):
    origin_criteria = list(set(address[address['시도 시군구'] == target]['행정동코드']))
    
    duration_arr = []
    distance_arr = []
    
    for idy, other in enumerate(daejun_shuttle):
        if idx == idy:
            od_duration = od_dist = 0
        else:
            dest_criteria = list(set(address[address['시도 시군구'] == other]['행정동코드']))
            tmp_df = od_df[
                od_df['origin_hdong_cd'].isin(origin_criteria) & od_df['dest_hdong_cd'].isin(dest_criteria)
            ]
            od_duration = tmp_df['od_duration_avg'].mean()
            od_dist = tmp_df['od_dist_avg'].mean()
        duration_arr.append(od_duration)
        distance_arr.append(od_dist)

    duration_table.append(duration_arr)
    distance_table.append(distance_arr)

In [103]:
for i in range(len(duration_table)):
    for j in duration_table[i]:
        print(f"{j:.2f}", end="     ")
    print() 

0.00     47.30     70.43     64.85     80.18     100.18     115.33     136.59     
62.36     0.00     38.53     42.92     55.87     80.37     99.36     113.92     
78.49     33.28     0.00     43.46     37.72     77.05     99.49     109.01     
70.13     37.72     49.40     0.00     50.15     78.24     87.24     111.99     
90.09     55.46     37.71     43.76     0.00     66.78     95.90     101.56     
100.05     76.48     75.72     74.79     63.45     0.00     61.66     59.71     
110.38     97.93     99.81     83.37     94.71     64.85     0.00     54.43     
132.54     102.68     108.21     101.83     96.98     64.41     54.10     0.00     


In [104]:
for i in range(len(distance_table)):
    for j in distance_table[i]:
        print(f"{j:.2f}", end="     ")
    print()

0.00     40119.17     63016.91     66187.98     77089.63     122123.70     142889.61     183927.22     
59516.73     0.00     22310.43     27028.69     37444.32     90192.69     118862.53     157314.19     
75954.88     17773.80     0.00     24094.24     20774.62     77790.67     113447.85     151477.61     
77138.29     23440.37     31390.31     0.00     32987.85     85423.80     97913.66     152123.38     
94547.19     34120.92     20395.79     25660.48     0.00     62673.19     102919.39     128662.81     
130583.28     86718.65     75305.95     81190.16     58805.41     0.00     66195.32     72484.04     
153002.83     115764.26     112807.82     99210.60     105239.72     69080.96     0.00     57398.01     
200733.67     145638.03     143661.91     147490.12     133625.24     86311.39     59351.75     0.00     


In [105]:
import numpy as np
import networkx as nx

# 정거장 후보지와 가중치
stations = {
    "세종특별자치시": 1,
    "대전광역시 유성구": 2,  # 가중치 2로 증가
    "대전광역시 서구": 2,    # 가중치 2로 증가
    "대전광역시 대덕구": 1,
    "대전광역시 중구": 1,
    "충청남도 금산군": 1,
    "충청북도 영동군": 2,    # 가중치 2로 증가
    "전라북도 무주군": 0,    # 목적지
}

# 그래프 생성
G = nx.DiGraph()

# 그래프에 엣지 추가
for i, from_station in enumerate(stations.keys()):
    for j, to_station in enumerate(stations.keys()):
        if i != j:
            G.add_edge(from_station, to_station, 
                       distance=distance_table[i][j], 
                       time=duration_table[i][j])

# 모든 경로 찾기
def find_all_routes(G, start, target, visited=None, path=None):
    if visited is None:
        visited = set()
    if path is None:
        path = []

    visited.add(start)
    path.append(start)

    if start == target:
        yield path.copy()
    else:
        for neighbor in G.neighbors(start):
            if neighbor not in visited:
                yield from find_all_routes(G, neighbor, target, visited, path)

    path.pop()
    visited.remove(start)

# 모든 출발 정거장에서의 경로 탐색
optimal_routes = []
max_stops = 0

for station in stations.keys():
    if station != "전라북도 무주군":  # 목적지가 아니면
        routes = list(find_all_routes(G, station, "전라북도 무주군"))
        
        # 바로 이동했을 때의 거리와 시간
        if "전라북도 무주군" in G[station]:  # 직접 경로가 있는 경우에만
            direct_distance = G[station]["전라북도 무주군"]['distance']
            direct_time = G[station]["전라북도 무주군"]['time']
            
            for route in routes:
                total_distance = sum(G[route[i]][route[i + 1]]['distance'] for i in range(len(route) - 1))
                total_time = sum(G[route[i]][route[i + 1]]['time'] for i in range(len(route) - 1))
                
                # 최적 경로 조건 확인
                if total_distance <= direct_distance * 1.3 and total_time <= direct_time * 1.3:
                    # 가장 많은 정거장 수 확인
                    if len(route) > max_stops:
                        max_stops = len(route)
                        optimal_routes = [(route, total_distance, total_time)]
                    elif len(route) == max_stops:
                        optimal_routes.append((route, total_distance, total_time))

# 결과 출력
if optimal_routes:
    for route, total_distance, total_time in optimal_routes:
        print(f"최적의 셔틀 노선: {route} | 총 거리: {total_distance} | 총 시간: {total_time}")
else:
    print("유효한 경로가 없습니다.")


최적의 셔틀 노선: ['세종특별자치시', '대전광역시 유성구', '전라북도 무주군'] | 총 거리: 197433.35808619377 | 총 시간: 161.2231652658421
최적의 셔틀 노선: ['세종특별자치시', '대전광역시 대덕구', '전라북도 무주군'] | 총 거리: 218311.3670601698 | 총 시간: 176.8416146179402
최적의 셔틀 노선: ['세종특별자치시', '충청남도 금산군', '전라북도 무주군'] | 총 거리: 194607.73787396512 | 총 시간: 159.89624306727757
최적의 셔틀 노선: ['세종특별자치시', '충청북도 영동군', '전라북도 무주군'] | 총 거리: 200287.6187426182 | 총 시간: 169.75847809126148
최적의 셔틀 노선: ['대전광역시 유성구', '대전광역시 서구', '전라북도 무주군'] | 총 거리: 173788.03796788197 | 총 시간: 147.53560501829148
최적의 셔틀 노선: ['대전광역시 유성구', '충청남도 금산군', '전라북도 무주군'] | 총 거리: 162676.73016826986 | 총 시간: 140.08587981230838
최적의 셔틀 노선: ['대전광역시 서구', '대전광역시 중구', '전라북도 무주군'] | 총 거리: 149437.4251537633 | 총 시간: 139.27864318395987
최적의 셔틀 노선: ['대전광역시 서구', '충청남도 금산군', '전라북도 무주군'] | 총 거리: 150274.70336384297 | 총 시간: 136.76739901734467
최적의 셔틀 노선: ['대전광역시 대덕구', '충청남도 금산군', '전라북도 무주군'] | 총 거리: 157907.8389282074 | 총 시간: 137.94978075658213
최적의 셔틀 노선: ['대전광역시 대덕구', '충청북도 영동군', '전라북도 무주군'] | 총 거리: 155311.6678968666 | 총 시간: 141.

In [98]:
# 인접 행렬 추출
adj_matrix = nx.adjacency_matrix(G).todense()

# pandas 데이터프레임으로 변환
df_adj_matrix = pd.DataFrame(adj_matrix, index=stations, columns=stations)

# 인접 행렬 예쁘게 출력
print("인접 행렬:")
print(df_adj_matrix)

인접 행렬:
           세종특별자치시  대전광역시 유성구  대전광역시 서구  대전광역시 대덕구  대전광역시 중구  충청남도 금산군  \
세종특별자치시          0          1         1          1         1         1   
대전광역시 유성구        1          0         1          1         1         1   
대전광역시 서구         1          1         0          1         1         1   
대전광역시 대덕구        1          1         1          0         1         1   
대전광역시 중구         1          1         1          1         0         1   
충청남도 금산군         1          1         1          1         1         0   
충청북도 영동군         1          1         1          1         1         1   
전라북도 무주군         1          1         1          1         1         1   

           충청북도 영동군  전라북도 무주군  
세종특별자치시           1         1  
대전광역시 유성구         1         1  
대전광역시 서구          1         1  
대전광역시 대덕구         1         1  
대전광역시 중구          1         1  
충청남도 금산군          1         1  
충청북도 영동군          0         1  
전라북도 무주군          1         0  
